# Analysis

In [584]:
%reload_ext autoreload
%autoreload 2

In [585]:
#export
from exp.nb_00 import *
from exp.nb_01 import *
from exp.nb_02 import *
import os
import shutil
import pickle
import re
import subprocess
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import numpy as np
from climatools.lblnew.export import vector_to_F77

In [586]:
param = LBLnewBestfitSWParam(**{'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'saw', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [3, 9], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)], 'cosz': 1, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0})
param

<class 'exp.nb_00.LBLnewBestfitSWParam'>
{'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'saw', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [3, 9], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)], 'cosz': 1, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}

In [587]:
PATH_FORTRAN = Path('/chia_cluster/home/jackyu/radiation/crdnew-sw/examples')

In [588]:
path = PATH_FORTRAN/'/'.join(sorted(f'{n}_{v}' for n, v in vars(param).items()))
runner = LBLnewBestfitSWRun(path, param)

In [490]:
runner.input_params()
proc = runner.build()
out, err = proc.communicate()

In [492]:
runner.run()

Job <435071> is submitted to queue <serial>.



In [494]:
! qstat -u jackyu

No matching job found
serial;  type=BATCH;  [ENABLED];  pri=35
3 run;   7 wait;


v4q;  type=BATCH;  [ENABLED];  pri=30
192 run;   0 wait;


v4qexp;  type=BATCH;  [ENABLED];  pri=30
144 run;   192 wait;


LCCR_Q;  type=BATCH;  [ENABLED];  pri=30
1152 run;   0 wait;


URBAN_Q;  type=BATCH;  [ENABLED];  pri=30
161 run;   448 wait;


orc;  type=BATCH;  [ENABLED];  pri=30
0 run;   0 wait;


RCEC_Q;  type=BATCH;  [ENABLED];  pri=30
96 run;   0 wait;


medium_priority;  type=BATCH;  [ENABLED];  pri=10
0 run;   0 wait;




## Set up analysis directory

In [589]:
PATH_ANALYSIS

PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw')

In [590]:
# We are going to test directly at a location inside the 'analysis_-_new_kdist_param_sw' repository, 
# because we will try to git-add and git-commit the executed notebook.
PATH = PATH_ANALYSIS/'/'.join(sorted(f'{n}_{v}' for n, v in vars(param).items()))
PATH

PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]')

In [591]:
PATH.mkdir(exist_ok=True, parents=True)

## Load the analysis notebook template

In [592]:
! pwd

/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev


In [593]:
os.chdir('/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev/')

In [609]:
# For now, we assume that the analysis notebook template resides in the current directory
# in which we're developing things: /chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev/.
with open('analysis_-_lblnew-bestfit-sw.ipynb') as f:
    analysis_nb = nbformat.read(f, as_version=nbformat.NO_CONVERT)

In [610]:
str(vars(runner.param))

"{'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'saw', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [3, 9], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)], 'cosz': 1, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}"

In [611]:
analysis_nb['cells'][2]['source'] = f'''PARAM = LBLnewBestfitSWParam(**{vars(runner.param)})'''
analysis_nb['cells'][2]

{'cell_type': 'code',
 'execution_count': 62,
 'metadata': {'collapsed': True},
 'outputs': [],
 'source': "PARAM = LBLnewBestfitSWParam(**{'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'saw', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [3, 9], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)], 'cosz': 1, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0})"}

In [613]:
Path(runner.path)

PosixPath('/chia_cluster/home/jackyu/radiation/crdnew-sw/examples/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]')

## Run the analysis notebook  
Useful: https://nbconvert.readthedocs.io/en/latest/execute_api.html

In [614]:
PATH, list(PATH.iterdir())

(PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]'),
 [PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]/analysis_-_lblnew-bestfit-sw.ipynb')])

In [615]:
ep = ExecutePreprocessor(timeout=600)
ep.preprocess(analysis_nb, {});

In [616]:
with open(PATH/'analysis_-_lblnew-bestfit-sw.ipynb', mode='w', encoding='utf-8') as f:
    nbformat.write(analysis_nb, f)

In [617]:
PATH

PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]')

## Git add the executed notebook

In [618]:
PATH, list(PATH.iterdir())

(PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]'),
 [PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]/analysis_-_lblnew-bestfit-sw.ipynb')])

In order to add the executed notebook to the 'analysis_-_lblnew-bestfit-sw' directory, we need to be under that directory at least. 

In [619]:
cwd = os.getcwd()
cwd

'/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev'

In [620]:
os.chdir(PATH)

In [621]:
! ls

analysis_-_lblnew-bestfit-sw.ipynb


In [622]:
proc = subprocess.Popen(['git', 'add', 'analysis_-_lblnew-bestfit-sw.ipynb'], 
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [623]:
out, err = proc.communicate()

In [624]:
print(out.decode(), err.decode())

In [625]:
! git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   analysis_-_lblnew-bestfit-sw.ipynb



## Git-commit the analysis notebook 
The commit message will contain the parameters for this particular case.

In [626]:
runner.param

<class 'exp.nb_00.LBLnewBestfitSWParam'>
{'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'saw', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [3, 9], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)], 'cosz': 1, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}

In [627]:
title = f"band{runner.param.band:02d} {runner.param.molecule} {runner.param.atmpro} cosz={runner.param.cosz} nf_refs={runner.param.ng_refs}"
title

'band09 h2o saw cosz=1 nf_refs=[3, 9]'

In [628]:
body = '\n'.join(sorted(f"{n} {v}" for n, v in vars(runner.param).items()))
print(body)

atmpro saw
band 9
commitnumber 79f2292
conc atmpro
cosz 1
dv 0.001
klin 3e-25
molecule h2o
ng_adju [-3, 0]
ng_refs [3, 9]
nv 10000
option_k_lookup 0
ref_pts [(30, 250), (300, 250)]
rsfc 0
tsfc None
vmax 14290
vmin 8200
wgt [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]


In [629]:
proc = subprocess.Popen(f'''git commit -m "{title}" -m "{body}"''', shell=True,
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [630]:
out, err = proc.communicate()

In [631]:
! git status

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [632]:
! git log -1

commit d84b5d412950c065555bf06e4924acbd6d4560a1 (HEAD -> master)
Author: Jack Yu <llacque@gmail.com>
Date:   Thu Jun 20 09:30:27 2019 +0800

    band09 h2o saw cosz=1 nf_refs=[3, 9]
    
    atmpro saw
    band 9
    commitnumber 79f2292
    conc atmpro
    cosz 1
    dv 0.001
    klin 3e-25
    molecule h2o
    ng_adju [-3, 0]
    ng_refs [3, 9]
    nv 10000
    option_k_lookup 0
    ref_pts [(30, 250), (300, 250)]
    rsfc 0
    tsfc None
    vmax 14290
    vmin 8200
    wgt [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]


## Get nbviewer url   
We want to be able to conveniently obtain the url that points to the uploaded jupyter notebook.

In [633]:
PRE_URL = Path('https://nbviewer.jupyter.org/github/qap')

In [634]:
gitname = 'analysis_-_new_kdist_param_sw'
if ('/' + gitname + '/') in str(PATH): _, suf = str(PATH).split(gitname)
else: suf = PATH
suf = 'blob/master' + suf
suf

'blob/master/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]'

In [635]:
PRE_URL/gitname/suf/'analysis_-_lblnew-bestfit-sw.ipynb'

PosixPath('https:/nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param_sw/blob/master/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]/analysis_-_lblnew-bestfit-sw.ipynb')

## Putting things together -- LBLnewBestfitSWAnalysis class

In [636]:
#export
class LBLnewBestfitSWAnalysis(object):
    def __init__(self, path, runner):
        self.path = Path(path)
        self.runner = runner
        self.path.mkdir(exist_ok=True, parents=True)
        
    def input_params(self):
        with open('analysis_-_lblnew-bestfit-sw.ipynb') as f:  # Need to set to stored absolute path
            nb = nbformat.read(f, as_version=nbformat.NO_CONVERT)
        nb['cells'][2]['source'] = f'''PARAM = LBLnewBestfitSWParam(**{vars(self.runner.param)})'''
        return nb
        
    def run(self):
        nb = self.input_params()
        ep = ExecutePreprocessor(timeout=600)
        ep.preprocess(nb, {})
        with open(self.path/'analysis_-_lblnew-bestfit-sw.ipynb', mode='w', encoding='utf-8') as f:
            nbformat.write(nb, f)
            
    def gitcommit(self):
        cwd = os.getcwd()
        os.chdir(self.path)
        try:
            proc = subprocess.Popen(['git', 'add', 'analysis_-_lblnew-bestfit-sw.ipynb'], 
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            out, err = proc.communicate()
            print('Git-add', out.decode(), err.decode())
            title = f"band{self.runner.param.band:02d} {self.runner.param.molecule} {self.runner.param.atmpro} cosz={self.runner.param.cosz} nf_refs={self.runner.param.ng_refs}"
            body = '\n'.join(sorted(f"{n} {v}" for n, v in vars(self.runner.param).items()))
            proc = subprocess.Popen(f'''git commit -m "{title}" -m "{body}"''', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            return proc
        finally: os.chdir(cwd)

    def nbviewer_url(self, gitname):
        '''
        gitname: str
            Git repository name.
        '''
        pre_url = Path('https://nbviewer.jupyter.org/github/qap')
        spath = str(self.path)
        if gitname in spath: 
            _, suf = spath.split(gitname)
        else: 
            suf = spath
        suf = ('blob/master' + suf).strip('/')
        return pre_url/gitname/suf/'analysis_-_lblnew-bestfit-sw.ipynb'

In [641]:
os.chdir(cwd)
! pwd

/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev


In [642]:
analysis = LBLnewBestfitSWAnalysis(path=PATH, runner=runner)

In [643]:
analysis.run()

In [644]:
proc = analysis.gitcommit()

Git-add  


In [645]:
out, err = proc.communicate()

In [646]:
print('Git-commit', out.decode(), err.decode())

Git-commit [master 5bc3a5b] band09 h2o saw cosz=1 nf_refs=[3, 9]
 1 file changed, 1759 insertions(+), 1759 deletions(-)
 rewrite atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]/analysis_-_lblnew-bestfit-sw.ipynb (75%)
 


In [647]:
analysis.nbviewer_url('analysis_-_new_kdist_param_sw')

PosixPath('https:/nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param_sw/blob/master/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]/analysis_-_lblnew-bestfit-sw.ipynb')

# Export notebook

In [648]:
! python notebook2script.py 03_analysis.ipynb

Converted 03_analysis.ipynb to exp/nb_03.py


# fin